In [29]:
import sys, os
sys.path.append(os.path.abspath(".."))
from data_process.data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor,
)

from diffusion_planner.data_process.roadblock_utils import (
  route_roadblock_correction
)


from diffusion_planner.data_process.agent_process import(sampled_tracked_objects_to_array_list,
                                                         sampled_static_objects_to_array_list,
                                                         agent_past_process,)
from diffusion_planner.data_process.map_process import(map_process,
                                                       get_neighbor_vector_set_map)

import numpy as np
import torch


In [30]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # mini db files

map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
device = torch.device("cuda:0")
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
del worker, builder, scenario_filter
processor = DataProcessor(scenarios, device)

Worker threads: 128


In [31]:
from diffusion_planner.data_process.utils import (
convert_absolute_quantities_to_relative,
_global_state_se2_array_to_local,
_global_velocity_to_local)


In [45]:
from nuplan.planning.training.preprocessing.utils.agents_preprocessing import(sampled_past_ego_states_to_tensor)
"""
ego past
"""
processor.scenario = scenarios[0]
print(processor.scenario.initial_ego_state)
processor.anchor_ego_state = np.array([processor.scenario.initial_ego_state.rear_axle.x, processor.scenario.initial_ego_state.rear_axle.y, processor.scenario.initial_ego_state.rear_axle.heading], \
                                      dtype=np.float64)

past_ego_states = processor.scenario.get_ego_past_trajectory(
    iteration=0, num_samples=processor.num_past_poses, time_horizon=processor.past_time_horizon
)
future_scenario_horizon = processor.scenario.duration_s.time_s - processor.past_time_horizon
scenario_num_samples = int(10 * future_scenario_horizon)
future_ego_states = processor.scenario.get_ego_future_trajectory(iteration=0, num_samples=scenario_num_samples, time_horizon= future_scenario_horizon)

sampled_ego_states = list(past_ego_states) + [processor.scenario.initial_ego_state] + list(future_ego_states)

# print(type(sampled_ego_states))
# print(len(sampled_ego_states))
sampled_ego_states_numpy = []
sampled_ego_states_numpy = np.array([(ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading, \
                                      ego_state.dynamic_car_state.rear_axle_velocity_2d.x, ego_state.dynamic_car_state.rear_axle_velocity_2d.y,\
                                      ego_state.dynamic_car_state.rear_axle_acceleration_2d.x, ego_state.dynamic_car_state.rear_axle_acceleration_2d.y ) for ego_state in sampled_ego_states], dtype = np.float64)
# print(sampled_ego_states_numpy.shape)

sampled_ego_states_numpy = np.pad(sampled_ego_states_numpy, ((0, processor.num_pred_poses), (0,0)), mode = 'constant', constant_values=np.nan)
# print(sampled_ego_states_numpy.shape)
local_sampled_past_ego_states = convert_absolute_quantities_to_relative(sampled_ego_states_numpy,processor.anchor_ego_state,'ego' )
print(local_sampled_past_ego_states.shape)
# processor.scenario = scenarios[0]
# """
# ego future
# """
# print(processor.scenario.duration_s)

# ego_state = processor.scenario.initial_ego_state
# anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], dtype=np.float64)
# ego_states_cat_numpy = ego_states_cat.numpy()
# print(ego_states_cat_numpy.shape)
# ego_state_cat_convert = convert_absolute_quantities_to_relative(ego_states_cat_numpy,anchor_ego_state,'ego' )

# print(type(ego_state_cat_convert))

(200, 7)


In [ ]:
# def process_tracked_objects(trakced_objects_in_all_frames, full_time_horizon, filter_agent_frames_threshold):

#   # 创建一个字典来存储所有障碍物的轨迹
#   object_trajectories = {}

#   # 遍历每一帧的 tracked_objects
#   for frame_idx, detection_frame in enumerate(trakced_objects_in_all_frames):
#     for obj in detection_frame.tracked_objects:
#         obj_token = obj.track_token  # 获取障碍物 ID
#         obj_state = np.array([obj.center.x, obj.center.y, obj.center.heading])  # 获取障碍物位置 (x, y)

#         # 如果该障碍物 ID 还未存储，则初始化
#         if obj_token not in object_trajectories:
#             object_trajectories[obj_token] = []

#         # 记录该障碍物在该帧的位置信息
#         object_trajectories[obj_token].append(obj_state)

#   keys_to_remove = [obj_token for obj_token, traj in object_trajectories.items() if len(traj) < filter_agent_frames_threshold]

#   for key in keys_to_remove:
#     del object_trajectories[key] #过滤少于filter_agent_frames_threshold帧的障碍物

#   # 统一轨迹长度，填充 NaN
#   for obj_token, traj in object_trajectories.items():
#     traj = np.array(traj)  # 转换为 NumPy 数组，形状为 (N, 2)
#     num_frames = traj.shape[0]

#     if num_frames < full_time_horizon:
#         # 如果轨迹长度小于 V，则用 NaN 填充
#         pad_size = full_time_horizon - num_frames
#         pad_array = np.full((pad_size, 3), np.nan)  # 生成 (pad_size, 2) 的 NaN 数组
#         object_trajectories[obj_token] =np.concatenate([traj, pad_array], axis=0) # 拼接填充
#     else:
#         # 如果轨迹长度大于 V，则裁剪
#         object_trajectories[obj_token] = traj[:full_time_horizon]



#   return object_trajectories

In [ ]:
def process_tracked_objects(tracked_objects_in_all_frames, full_time_horizon, filter_agent_frames_threshold):
    """
    处理自动驾驶中的障碍物数据，将其转换为 (T, N, 3) 形状的 NumPy 数组
    :param tracked_objects_in_all_frames: 包含所有时间步的障碍物检测信息
    :param full_time_horizon: 需要对齐的时间长度（T）
    :param filter_agent_frames_threshold: 过滤掉轨迹长度小于该值的障碍物
    :return: 形状为 (T, N, 3) 的 NumPy 数组，N 为所有时间步中最大障碍物数
    """

    # 创建一个字典存储每个障碍物的轨迹
    object_trajectories = {}

    # 遍历每一帧的 tracked_objects
    for frame_idx, detection_frame in enumerate(tracked_objects_in_all_frames):
        for obj in detection_frame.tracked_objects:
            obj_token = obj.track_token  # 获取障碍物唯一 ID
            obj_state = np.array([obj.center.x, obj.center.y, obj.center.heading])  # 提取 (x, y, heading)

            # 如果该障碍物 ID 还未存储，则初始化
            if obj_token not in object_trajectories:
                object_trajectories[obj_token] = np.full((full_time_horizon, 3), np.nan)

            # 记录该障碍物在该帧的位置信息
            object_trajectories[obj_token][frame_idx] = obj_state

    # **第一步：过滤掉轨迹长度小于 filter_agent_frames_threshold 的障碍物**
    object_trajectories = {
        obj_token: traj for obj_token, traj in object_trajectories.items()
        if np.sum(~np.isnan(traj[:, 0])) >= filter_agent_frames_threshold
    }

    return object_trajectories

In [85]:
"""
agent future
"""
filter_agent_frames_threshold = 10
neighbors = processor.scenario.initial_tracked_objects
neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=processor.past_time_horizon, num_samples = int( 10 * processor.past_time_horizon)))
track_objs = neighbor_past[0].tracked_objects # center, track_token
print(len(neighbor_past))

neighbor_future = list(processor.scenario.get_future_tracked_objects(iteration= 0, time_horizon=future_scenario_horizon, num_samples =int( 10 * future_scenario_horizon)))
neighbor_states = neighbor_past +[neighbors]+ neighbor_future
print(len(neighbor_states))

padded_neighbors= process_tracked_objects(neighbor_states, 201, filter_agent_frames_threshold)
print(len(padded_neighbors))
padded_neighbors_array = np.array(list(padded_neighbors.values()))
print(type(padded_neighbors_array))
print(padded_neighbors_array.shape)
# padded_neighbors_array = np.swapaxes(padded_neighbors_array, 0, 1)
local_padded_neighbor_states_array = []
for i in range(len(padded_neighbors_array)):
  local_padded_neighbor_states_array.append(np.array(convert_absolute_quantities_to_relative(padded_neighbors_array[i, :, :], processor.anchor_ego_state, 'pos_heading_only')))
# local_padded_neighbor_states_array = np.array(convert_absolute_quantities_to_relative(padded_neighbors_array[0, :, :], processor.anchor_ego_state, 'pos_heading_only'))
local_padded_neighbor_states_array = np.array(local_padded_neighbor_states_array)
print(type(local_padded_neighbor_states_array))
local_padded_neighbor_agents_past_array = local_padded_neighbor_states_array[:, :20+ 1, :]

# print(horizon)

20
150
103
<class 'numpy.ndarray'>
(103, 201, 3)
<class 'numpy.ndarray'>


In [ ]:



from nuplan.planning.training.preprocessing.utils.agents_preprocessing import EgoInternalIndex, AgentInternalIndex


In [ ]:
from diffusion_planner.data_process.data_processor import DataProcessor
print(len(neighbor_states))

# padded_neighbor_states = list(padded_neighbors)
print(len(padded_neighbors))
object_traj_list = []
# for key, value in padded_neighbors.items():
#   object_traj_list.append(value)

local_object_traj_list = []
# print(len(padded_neighbor_states))

# local_object_traj_list = np.array(convert_absolute_quantities_to_relative(object_traj_list, anchor_ego_state, 'pos_heading_only'))

In [ ]:
def process_tracked_objects(tracked_objects_in_all_frames, full_time_horizon: int):

  # 创建一个字典来存储所有障碍物的轨迹
  object_trajectories = {}

  # 遍历每一帧的 tracked_objects
  for frame_idx, detection_frame in enumerate(tracked_objects_in_all_frames):
      for obj in detection_frame.tracked_objects:
          obj_token = obj.track_token  # 获取障碍物 ID
          obj_state = np.array([obj.center.x, obj.center.y, obj.center.heading])  # 获取障碍物位置 (x, y)

          # 如果该障碍物 ID 还未存储，则初始化
          if obj_token not in object_trajectories:
              object_trajectories[obj_token] = []

          # 记录该障碍物在该帧的位置信息
          object_trajectories[obj_token].append(obj_state)

  # 统一轨迹长度，填充 NaN
  for obj_token, traj in object_trajectories.items():
      traj = np.array(traj)  # 转换为 NumPy 数组，形状为 (N, 2)
      num_frames = traj.shape[0]

      if num_frames < full_time_horizon:
          # 如果轨迹长度小于 V，则用 NaN 填充
          pad_size = full_time_horizon - num_frames
          pad_array = np.full((pad_size, 3), np.nan)  # 生成 (pad_size, 2) 的 NaN 数组
          object_trajectories[obj_token] = np.vstack([traj, pad_array])  # 拼接填充
      else:
          # 如果轨迹长度大于 V，则裁剪
          object_trajectories[obj_token] = traj[:full_time_horizon]

  return object_trajectories


In [ ]:
from diffusion_planner.data_process.utils import (
convert_absolute_quantities_to_relative,
_global_state_se2_array_to_local,
_global_velocity_to_local)


from nuplan.planning.training.preprocessing.utils.agents_preprocessing import EgoInternalIndex, AgentInternalIndex

In [ ]:
default_pred_horizon =5.0
filter_
object_traj = process_tracked_objects(neighbor_states, int(1 + (processor.past_time_horizon + future_scenario_horizon + default_pred_horizon)*10), )
# object_traj_tensor = torch.full(object_traj.__len__)
print(type(object_traj))
# for key, value in object_traj.items():
#   object_traj_list.append(value)

print(len(object_traj))
local_object_traj_list = []
print(processor.scenario.initial_ego_state)
print(EgoInternalIndex.x())
ego_state = processor.scenario.initial_ego_state

anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], dtype=np.float64)
# local_coords_agent_states.append(convert_absolute_quantities_to_relative(object_traj_list, processor.anchor_ego_state, 'pos_heading_only'))
local_object_traj_list = convert_absolute_quantities_to_relative(object_traj_list, anchor_ego_state, 'pos_heading_only')
print(len(local_object_traj_list))
print(local_object_traj_list[0].shape)


In [ ]:
print(type(local_object_traj_list))
ar = np.array(local_object_traj_list)
print(ar.shape)
local_padded_neighbor_agents_past = ar[:, :21, ...]
print(local_padded_neighbor_agents_past.shape)

In [ ]:
def process_ego_state(ego_states_in_all_frames, full_time_horizon: int):

  output = torch.zeros((len(ego_states_in_all_frames), 3), dtype=torch.float32)
  for i in range(0, len(ego_states_in_all_frames), 1):
    output[i, 0] = ego_states_in_all_frames[i].rear_axle.x
    output[i, 1] = ego_states_in_all_frames[i].rear_axle.y
    output[i, 2] = ego_states_in_all_frames[i].rear_axle.heading


  return output

In [ ]:
ego_traj = list(future_ego_states) + sampled_past_ego_states
print(len(ego_traj))
ego_traj = process_ego_state(ego_traj, int(1 + (processor.past_time_horizon + future_scenario_horizon + default_pred_horizon)*10) )
print(ego_traj.shape)

In [ ]:
from diffusion_planner.data_process.utils import (
convert_absolute_quantities_to_relative,
_global_state_se2_array_to_local,
_global_velocity_to_local)




In [ ]:
import os

# 处理时间格式

save_path = "/cailiu2/Diffusion-Planner/data/processed"
# 生成文件名
file_name = f"{processor.scenario.log_name}_{processor.scenario.token}_{processor.scenario.scenario_type}.pt"

# 确保文件名合法（移除特殊字符）
file_name = file_name.replace(":", "_").replace("/", "_").replace(" ", "_")

# 指定存放路径
os.makedirs(save_path, exist_ok=True)  # 确保目录存在

# 完整文件路径
file_path = os.path.join(save_path, file_name)
torch.save(data, file_path)


In [ ]:
import torch
print(torch.cuda.device_count())  # 如果为 True，表示有 CUDA 可用；如果为 False，则没有 CUDA 可用
print(torch.__version__)